In [3]:
import sys
import os
from dotenv import load_dotenv
load_dotenv()

sys.path.append(os.environ["WORK_DIR"])

In [4]:
from concurrent.futures import ThreadPoolExecutor
from typing import List, Dict, Callable
from util import llm_call, extract_xml


def route(input: str, routes: Dict[str, str]) -> str:
    """Route input to specialized prompt using content classification."""
    # First determine appropriate route using LLM with chain-of-thought
    print(f"\nAvailable routes: {list(routes.keys())}")
    selector_prompt = f"""
    Analyze the input and select the most appropriate CRUD operation should be done from these options: {list(routes.keys())}.
    We gonna perform the operation in the database for that store the relevant user information.
    First explain your reasoning, then provide your selection in this XML format:

    <reasoning>
    Brief explanation of why this ticket should be routed to a specific operation.
    Consider key terms, user intent, and urgency level.
    </reasoning>

    <selection>
    The chosen team operation
    </selection>

    Input: {input}""".strip()
    
    route_response = llm_call(selector_prompt)
    reasoning = extract_xml(route_response, 'reasoning')
    route_key = extract_xml(route_response, 'selection').strip().lower()
    
    print("Routing Analysis:")
    print(reasoning)
    print(f"\nSelected route: {route_key}")
    
    # Process input with selected specialized prompt
    selected_prompt = routes[route_key]
    return llm_call(f"{selected_prompt}\nInput: {input}")

In [ ]:
support_routes = {
    "create": """You are a CRUD operations specialist. Follow these guidelines:
    1. Always start with "CRUD Support Response:".
    2. Validate and confirm format 
    2. Confirm  and validate that all mandatory fields for creation are provided and in the right format.
    3. Validate the format and consistency of the new data.
    4. Write a SQL query to create as new data as per given information
    5. Add the new record and return the sql query only.
    
    Keep responses professional but friendly.
    
    Input: """,
    
    "read": """You are a technical support engineer. Follow these guidelines:
    1. Always start with "CRUD Support Response:".
    2. Confirm that query parameters or record identifiers are present.
    3. Write a query to fetch the details from the db.
    4. Retrieve the record and return the sql query only.
    
    Maintain a serious, security-focused tone.
    Input: """,
    
    "update": """You are a CRUD operations specialist. Follow these guidelines:
    1. Always start with "CRUD Support Response:".
    2. Write a sql query to fetch the specific data.
    3. Validate the new data against required formats and constraints.
    4. Write a query to fetch the details from the db.
    5. Update the record and return the sql query only.
    
    Maintain a serious, security-focused tone.
    
    Input: """,
    
    "delete": """You are a CRUD operations specialist. Follow these guidelines:
    1. Always start with "CRUD Support Response:".
    2. Write sql query to fetch the specific data
    3. Confirm the record exists.
    4. Validate that deletion is permitted and safe.
    4. Write a query to delete the details from the db.
    5. delete the record and return the sql query only.
    
    Maintain a serious, security-focused tone.
    
    Input: """
}

# Test with different support tickets
tickets_crud = [
    """Subject: Create New Employee Record
Message: Hi Support,
I need to add a new employee record to our HR system. The details are:
- Name: Emma Wilson
- Employee ID: EMP1024
- Department: Marketing
- Email: emma.wilson@example.com
Please verify that all required fields are provided before proceeding with the creation.
Thanks,
HR Team""",
    
    """Subject: Read Request - Retrieve Customer Order Details
Message: Hello,
I would like to retrieve the details for Order ID #ORD12345, including the current status and shipping information.
Kindly confirm if additional parameters are needed to perform this query.
Regards,
Customer Service""",
    
    """Subject: Update Record - Modify Product Pricing
Message: Hi,
I need to update the pricing for Product ID PROD56789. The new price should be $99.99 instead of the current one.
Please verify the record details and ensure that the updated pricing adheres to our policies before making the change.
Thank you,
Product Management""",
    
    """Subject: Delete Request - Remove Inactive User Account
Message: Hello,
I would like to request the deletion of an inactive user account (User ID: U78910). 
Please check that this account has no active subscriptions or dependencies before proceeding with deletion.
Awaiting your confirmation.
Best regards,
Admin Team"""
]

print("Processing support tickets...\n")
for i, ticket in enumerate(tickets_crud, 1):
    print(f"\nTicket {i}:")
    print("-" * 40)
    print(ticket)
    print("\nResponse:")
    print("-" * 40)
    response = route(ticket, support_routes)
    print(response)

Processing support tickets...


Ticket 1:
----------------------------------------
Subject: Create New Employee Record
Message: Hi Support,
I need to add a new employee record to our HR system. The details are:
- Name: Emma Wilson
- Employee ID: EMP1024
- Department: Marketing
- Email: emma.wilson@example.com
Please verify that all required fields are provided before proceeding with the creation.
Thanks,
HR Team

Response:
----------------------------------------

Available routes: ['create', 'read', 'update', 'delete']
Routing Analysis:

The subject and message clearly indicate the intent to add a new employee record to the HR system. The use of the word "Create" in the subject and "add a new employee record" in the message suggests that this is a request to insert new data into the database. The urgency level is not explicitly stated, but the request is straightforward and typical for HR operations. The key terms "create," "add," and "new employee record" align with the 'create' oper